In [12]:
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
import argparse
import pandas as pd

In [2]:
# best model path
bast_model_path = 'saved/models/ConstellationClassification/1120_131506/model_best.pth'
# 读入配置文件
parser = argparse.ArgumentParser(description='PyTorch Template')
parser.add_argument('-c', '--config', default=None, type=str,
                  help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=bast_model_path,
                    type=str, help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')
args = parser.parse_args(args=['--resume', bast_model_path, '--device', 'all'])
config = ConfigParser.from_args(args, jupyter=True)


In [14]:
# 设置logger
logger = config.get_logger('test')
# 获取dataloader
data_loader = getattr(module_data, config['data_loader']['type'])(
    batch_size=1,
    shuffle=False,
    validation_split=0.0,
    training=False,
    num_workers=2
)
# 设置model
model = config.init_obj('arch', module_arch)
# logger.info(model)
# 设置损失函数等
loss_fn = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']
if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

total_loss = 0.0
total_metrics = torch.zeros(len(metric_fns))

Loading checkpoint: saved/models/ConstellationClassification/1120_131506/model_best.pth ...


/home/ubuntu/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
results_list = []
with torch.no_grad():
    for i, (data, target, noisy_value) in enumerate(tqdm(data_loader)):
        result = []
        data, target = data.to(device), target.to(device)
        output = model(data)
        #
        # save sample images, or do something with output here
        #
        # computing loss, metrics on test set
        loss = loss_fn(output, target)
        batch_size = data.shape[0]
        total_loss += loss.item() * batch_size
        result.append(target[0].item())
        result.append(noisy_value[0])
        result.append(loss.item() * batch_size)
        for i, metric in enumerate(metric_fns):
            metric_result = metric(output, target)
            total_metrics[i] += metric_result * batch_size
            result.append(metric_result)
        results_list.append(result)

# 转化为dataframe
results_df = pd.DataFrame(results_list, columns=['label', 'noisy_value', 'loss', 'acc', 'top-k acc'])
            
        

100%|██████████| 600/600 [00:03<00:00, 165.93it/s]


In [10]:
n_samples = len(data_loader.sampler)
log = {'loss': total_loss / n_samples}
log.update({
    met.__name__: total_metrics[i].item() / n_samples for i, met in enumerate(metric_fns)
})
logger.info(log)

{'loss': -1266.937238260905, 'accuracy': 0.955, 'top_k_acc': 1.0}


In [22]:
result

[4, 7.0, -1074.69921875, 0.0, 1.0]